In [1]:
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark.sql.types import StructField, StructType, StringType, IntegerType
import pyspark.sql.functions as f
import re
from collections import Counter

In [2]:
sc = SparkSession.builder.getOrCreate()

In [3]:
df = sc.read.text("gs://diobucket1/sherlock.txt")
df.show(5)

+--------------------+
|               value|
+--------------------+
|The Project Guten...|
|                    |
|This eBook is for...|
|most other parts ...|
|whatsoever. You m...|
+--------------------+
only showing top 5 rows



In [4]:
#spark dataframe to list
list_livro = list(df.select('value').toPandas()['value'])

In [5]:
#Cria uma lista com cada palavra (sem caracteres especiais) existente no .txt
resultado = []
for elemento in range(len(list_livro)):
    splita_linha = list_livro[elemento].split(' ')
    for cada_palavra in range (len(splita_linha)):
        if re.match(r'[\w]+',splita_linha[cada_palavra]):
            resultado.append(re.sub(r'[\W]+',"",splita_linha[cada_palavra]).upper()) 
#print(resultado)           

In [6]:
#metodo 1 collections Counter
#conta a quantidade de vezes que a palavra aparece na lista
contaPalavra = Counter(resultado)

#for palavras in contaPalavra.items(): print("{}\t{}".format(*palavras))

In [7]:
# python list -> spark dataframe
#novo schema dataframe
schema = StructType([
    StructField('palavra', StringType(), True),
    StructField('qnt', IntegerType(), True)
])
# lista -> rdd
rdd = sc.sparkContext.parallelize(contaPalavra.items())

# Novo data frame
df = sc.createDataFrame(rdd,schema)
df.show()

+----------+----+
|   palavra| qnt|
+----------+----+
|       THE|5709|
|   PROJECT|  84|
| GUTENBERG|  32|
|     EBOOK|  12|
|        OF|2770|
|ADVENTURES|  10|
|  SHERLOCK| 100|
|    HOLMES| 455|
|        BY| 359|
|    ARTHUR|  18|
|     CONAN|   3|
|     DOYLE|   3|
|      THIS| 493|
|        IS|1127|
|       FOR| 740|
|       USE|  45|
|    ANYONE|  29|
|  ANYWHERE|   4|
|        IN|1796|
|    UNITED|  18|
+----------+----+
only showing top 20 rows



In [8]:
#valores diferentes do mrjob pois estava testando regras diferentes em reg_exp
oi = df.orderBy(['qnt'],ascending=[0])
oi.head(10)

[Row(palavra='THE', qnt=5709),
 Row(palavra='AND', qnt=2935),
 Row(palavra='OF', qnt=2770),
 Row(palavra='TO', qnt=2744),
 Row(palavra='A', qnt=2651),
 Row(palavra='I', qnt=2603),
 Row(palavra='IN', qnt=1796),
 Row(palavra='THAT', qnt=1678),
 Row(palavra='IT', qnt=1566),
 Row(palavra='HE', qnt=1420)]

In [9]:
df1columa= df.select(f.concat_ws(' = ',*df.columns).alias('palavras e quantidade'))

In [10]:
df1columa.show()

+---------------------+
|palavras e quantidade|
+---------------------+
|           THE = 5709|
|         PROJECT = 84|
|       GUTENBERG = 32|
|           EBOOK = 12|
|            OF = 2770|
|      ADVENTURES = 10|
|       SHERLOCK = 100|
|         HOLMES = 455|
|             BY = 359|
|          ARTHUR = 18|
|            CONAN = 3|
|            DOYLE = 3|
|           THIS = 493|
|            IS = 1127|
|            FOR = 740|
|             USE = 45|
|          ANYONE = 29|
|         ANYWHERE = 4|
|            IN = 1796|
|          UNITED = 18|
+---------------------+
only showing top 20 rows



In [11]:
#Salva no bucket em .txt
#df1columa.coalesce(1).write.format("text").option("header", "false").mode("append").save("gs://diobucket1/sherlockresultado")